# Single Latent Evolution: Watch a Shape Form

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jtooates/blind_lm/blob/main/experiments/single_latent_evolution.ipynb)

This notebook lets you watch a **single latent image** evolve step-by-step under different loss functions.

**Purpose**: Understand how each loss affects the visual appearance during optimization.

**Workflow**:
1. Configure loss weights
2. Start with random noise
3. Watch it transform into a shape frame-by-frame

## Setup

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

print(f"PyTorch version: {torch.__version__}")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

## Loss Functions

In [ ]:
def compute_losses(latent):
    """
    Compute all loss components for a single latent [H, W].
    Returns dict of individual losses.
    """
    # Add batch dimension for compatibility
    latent_batch = latent.unsqueeze(0)  # [1, H, W]
    losses = {}
    
    # 1. Sparsity: encourages most pixels → 0 (background)
    losses['sparsity'] = torch.mean(torch.abs(latent))
    
    # 2. Binary: pushes pixels toward -1 or +1 (sharp black/white)
    distance_from_binary = torch.min(
        (latent - 1.0)**2,
        (latent + 1.0)**2
    )
    losses['binary'] = torch.mean(distance_from_binary)
    
    # 3. Total Variation: smoothness within regions (penalizes edges)
    dx = torch.abs(latent[1:, :] - latent[:-1, :])
    dy = torch.abs(latent[:, 1:] - latent[:, :-1])
    losses['tv'] = torch.mean(dx) + torch.mean(dy)
    
    # 4. Object Size: enforces target ratio of bright pixels
    threshold = 0.5
    binary_mask = (latent > threshold).float()
    bright_ratio = torch.mean(binary_mask)
    target_ratio = 0.25
    losses['object_size'] = (bright_ratio - target_ratio)**2
    
    # 5. Magnitude: prevents collapse to near-zero values
    magnitude = torch.mean(torch.abs(latent))
    min_magnitude = 0.3
    losses['magnitude'] = torch.relu(min_magnitude - magnitude)
    
    # 6. Autocorrelation: encourages spatially connected blobs
    dx_corr = latent[1:, :] * latent[:-1, :]
    dy_corr = latent[:, 1:] * latent[:, :-1]
    autocorr = torch.mean(dx_corr) + torch.mean(dy_corr)
    losses['coherence_autocorr'] = -autocorr  # Negative because we want high autocorr
    
    # 7. Perimeter-to-Area: compact blobs vs scattered pixels
    temperature = 0.1
    binary_soft = torch.sigmoid((latent - threshold) / temperature)
    dx_edge = torch.abs(binary_soft[1:, :] - binary_soft[:-1, :])
    dy_edge = torch.abs(binary_soft[:, 1:] - binary_soft[:, :-1])
    edge_length = torch.sum(dx_edge) + torch.sum(dy_edge)
    area = torch.sum(binary_soft) + 1e-6
    losses['coherence_perimeter'] = edge_length / area
    
    # 8. Morphological: penalizes scattered pixels
    latent_4d = latent_batch.unsqueeze(1)  # [1, 1, H, W]
    pooled = F.max_pool2d(latent_4d, kernel_size=3, stride=1, padding=1)
    unpooled = -F.max_pool2d(-pooled, kernel_size=3, stride=1, padding=1)
    unpooled = unpooled.squeeze()
    losses['coherence_morphological'] = torch.mean((latent - unpooled) ** 2)
    
    return losses

print("✓ Loss functions defined")

## 1. Configure Loss Weights

**Adjust these to control what the image looks like!**

Set any weight to `0` to disable that loss.

In [ ]:
# ============================================================================
# LOSS WEIGHTS - EXPERIMENT WITH THESE!
# ============================================================================

weights = {
    # Sparsity: encourages most pixels → 0 (background)
    # Higher = more black background, fewer active pixels
    'sparsity': 0.1,
    
    # Binary: pushes pixels toward -1 or +1 (sharp black/white)
    # Higher = sharper boundaries, less gray
    'binary': 1.0,
    
    # Total Variation: smoothness within regions (penalizes edges)
    # Higher = smoother blobs, fewer sharp edges
    'tv': 0.05,
    
    # Object Size: enforces target ratio of bright pixels (~25%)
    # Higher = stronger enforcement of target coverage
    'object_size': 2.0,
    
    # Magnitude: prevents collapse to near-zero values
    # Higher = forces stronger signals
    'magnitude': 5.0,
    
    # Coherence (choose ONE or mix):
    'coherence_autocorr': 2.0,        # Spatial autocorrelation (recommended)
    'coherence_perimeter': 0.0,       # Perimeter-to-area ratio
    'coherence_morphological': 0.0,   # Morphological smoothness
}

# ============================================================================
# OPTIMIZATION SETTINGS
# ============================================================================

num_steps = 500              # Total optimization steps
snapshot_interval = 25       # Save visualization every N steps
learning_rate = 0.02         # Adam learning rate
image_size = 32              # Latent is 32x32
random_seed = 42             # For reproducibility (None = random)

# ============================================================================

print("Configuration:")
print(f"  Total steps: {num_steps}")
print(f"  Snapshots every {snapshot_interval} steps")
print(f"  Expected snapshots: {num_steps // snapshot_interval + 1}")
print(f"  Learning rate: {learning_rate}")
print(f"\nActive losses:")
for name, weight in weights.items():
    if weight > 0:
        print(f"  {name:25s}: {weight:.2f}")

## 2. Initialize Single Latent

Start with Gaussian noise + a blob seed

In [ ]:
# Set random seed if specified
if random_seed is not None:
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)

# Initialize single latent [H, W]
latent = torch.randn(image_size, image_size) * 0.1

# Add a random blob to break symmetry
for _ in range(2):
    y = np.random.randint(5, image_size - 5)
    x = np.random.randint(5, image_size - 5)
    size = np.random.randint(3, 7)
    
    yy, xx = torch.meshgrid(
        torch.arange(image_size) - y,
        torch.arange(image_size) - x,
        indexing='ij'
    )
    blob = torch.exp(-(yy**2 + xx**2) / (2 * size**2))
    latent += blob * np.random.uniform(0.5, 1.5)

latent = latent.to(device)
latent.requires_grad_(True)

# Visualize initial state
plt.figure(figsize=(5, 5))
plt.imshow(latent.detach().cpu().numpy(), cmap='gray', vmin=-1.5, vmax=1.5)
plt.title('Initial Latent (Step 0)', fontsize=14, fontweight='bold')
plt.colorbar(label='Pixel Value')
plt.axis('off')
plt.tight_layout()
plt.show()

print(f"✓ Initialized latent: {latent.shape}")
print(f"  Min value: {latent.min().item():.3f}")
print(f"  Max value: {latent.max().item():.3f}")
print(f"  Mean value: {latent.mean().item():.3f}")

## 3. Run Optimization with Snapshots

Watch the latent evolve step by step!

In [ ]:
# Reset latent (re-run cell above if you want new random init)
optimizer = torch.optim.Adam([latent], lr=learning_rate)

# Storage for snapshots
snapshots = []
snapshot_steps = []
loss_history = []

# Save initial state
snapshots.append(latent.detach().cpu().numpy().copy())
snapshot_steps.append(0)

print("Starting optimization...\n")

for step in range(num_steps):
    optimizer.zero_grad()
    
    # Compute all losses
    losses = compute_losses(latent)
    
    # Build weighted total loss
    total_loss = sum(losses[name] * weights[name] for name in losses if name in weights)
    
    # Optimize
    total_loss.backward()
    optimizer.step()
    
    # Clamp to valid range
    with torch.no_grad():
        latent.clamp_(-1.5, 1.5)
    
    loss_history.append(total_loss.item())
    
    # Save snapshot
    if (step + 1) % snapshot_interval == 0:
        snapshots.append(latent.detach().cpu().numpy().copy())
        snapshot_steps.append(step + 1)
        print(f"Step {step + 1:3d}/{num_steps}: Loss = {total_loss.item():.4f}")

# Save final state if not already saved
if num_steps % snapshot_interval != 0:
    snapshots.append(latent.detach().cpu().numpy().copy())
    snapshot_steps.append(num_steps)

print(f"\n✓ Optimization complete!")
print(f"  Captured {len(snapshots)} snapshots")
print(f"  Final loss: {loss_history[-1]:.4f}")
print(f"  Initial loss: {loss_history[0]:.4f}")

## 4. Visualize Evolution

See all snapshots in a grid

In [ ]:
# Calculate grid size
num_snapshots = len(snapshots)
cols = min(5, num_snapshots)
rows = (num_snapshots + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))
if rows == 1:
    axes = axes.reshape(1, -1)
if cols == 1:
    axes = axes.reshape(-1, 1)

for idx, (snapshot, step) in enumerate(zip(snapshots, snapshot_steps)):
    row = idx // cols
    col = idx % cols
    
    ax = axes[row, col]
    ax.imshow(snapshot, cmap='gray', vmin=-1.5, vmax=1.5)
    ax.set_title(f'Step {step}', fontsize=10)
    ax.axis('off')

# Hide unused subplots
for idx in range(num_snapshots, rows * cols):
    row = idx // cols
    col = idx % cols
    axes[row, col].axis('off')

plt.suptitle('Latent Evolution Over Time', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nEvolution summary:")
print(f"  Initial → Final: {snapshot_steps[0]} → {snapshot_steps[-1]} steps")
print(f"  You can see the transformation from random noise to structured shape!")

## 5. Plot Loss Curve

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(loss_history, linewidth=2)

# Mark snapshot points
for step in snapshot_steps[1:]:  # Skip initial
    if step < len(loss_history):
        plt.axvline(x=step, color='red', alpha=0.3, linestyle='--', linewidth=1)

plt.xlabel('Optimization Step', fontsize=12)
plt.ylabel('Total Loss', fontsize=12)
plt.title('Loss During Optimization (red lines = snapshots)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Loss reduction: {(loss_history[0] - loss_history[-1]) / abs(loss_history[0]) * 100:.1f}%")

## 6. Side-by-Side Comparison

See initial vs final clearly

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Initial
axes[0].imshow(snapshots[0], cmap='gray', vmin=-1.5, vmax=1.5)
axes[0].set_title(f'Initial (Step {snapshot_steps[0]})', fontsize=14, fontweight='bold')
axes[0].axis('off')

# Final
axes[1].imshow(snapshots[-1], cmap='gray', vmin=-1.5, vmax=1.5)
axes[1].set_title(f'Final (Step {snapshot_steps[-1]})', fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.suptitle('Transformation: Noise → Shape', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Print statistics
print("Initial latent:")
print(f"  Min: {snapshots[0].min():.3f}")
print(f"  Max: {snapshots[0].max():.3f}")
print(f"  Mean: {snapshots[0].mean():.3f}")
print(f"  Std: {snapshots[0].std():.3f}")

print("\nFinal latent:")
print(f"  Min: {snapshots[-1].min():.3f}")
print(f"  Max: {snapshots[-1].max():.3f}")
print(f"  Mean: {snapshots[-1].mean():.3f}")
print(f"  Std: {snapshots[-1].std():.3f}")

## 7. Analyze Individual Loss Components

See how each loss evolves

In [ ]:
# Recompute losses at each snapshot
loss_components_history = {name: [] for name in weights.keys()}

for snapshot in snapshots:
    snapshot_tensor = torch.tensor(snapshot, dtype=torch.float32)
    losses = compute_losses(snapshot_tensor)
    for name in weights.keys():
        loss_components_history[name].append(losses[name].item())

# Plot active losses
active_losses = {name: values for name, values in loss_components_history.items() 
                if weights[name] > 0}

if active_losses:
    fig, axes = plt.subplots(len(active_losses), 1, figsize=(10, 3 * len(active_losses)))
    if len(active_losses) == 1:
        axes = [axes]
    
    for ax, (name, values) in zip(axes, active_losses.items()):
        ax.plot(snapshot_steps, values, linewidth=2, marker='o')
        ax.set_xlabel('Step', fontsize=10)
        ax.set_ylabel('Loss Value', fontsize=10)
        ax.set_title(f'{name} (weight={weights[name]:.2f})', fontsize=12, fontweight='bold')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No active losses to plot (all weights are 0)")

## Experiments to Try

Go back to the "Configure Loss Weights" cell and try:

### 1. Pure Coherence (No Other Losses)
```python
weights = {
    'sparsity': 0.0,
    'binary': 0.0,
    'tv': 0.0,
    'object_size': 0.0,
    'magnitude': 0.0,
    'coherence_autocorr': 2.0,
}
```
**Expected**: Smooth blobs, but may drift toward all positive or all negative

### 2. Coherence + Magnitude (Prevent Collapse)
```python
weights = {
    'magnitude': 5.0,
    'coherence_autocorr': 2.0,
}
```
**Expected**: Blobs with guaranteed signal strength

### 3. Full Stack (Recommended)
```python
weights = {
    'sparsity': 0.1,
    'binary': 1.0,
    'tv': 0.05,
    'object_size': 2.0,
    'magnitude': 5.0,
    'coherence_autocorr': 2.0,
}
```
**Expected**: Clean, compact blobs on sparse background

### 4. Compare Coherence Types
Try each coherence loss individually to see the difference:
- `coherence_autocorr`: Smooth, connected
- `coherence_perimeter`: Compact, circular
- `coherence_morphological`: Robust to smoothing